In [1]:
from flask import Flask, request, jsonify
import requests
import gtfs_realtime_pb2  # Ensure this file is generated


In [2]:
# URL for MTA's GTFS-Realtime data
MTA_FEED_URL = 'https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs-ace'

def fetch_mta_alerts():
    feed = gtfs_realtime_pb2.FeedMessage()
    try:
        response = requests.get(MTA_FEED_URL)
        response.raise_for_status()
        feed.ParseFromString(response.content)
    except requests.RequestException as e:
        return {"error": str(e)}

    alerts = []
    for entity in feed.entity:
        if entity.HasField('alert'):
            alert_text = entity.alert.header_text.translation[0].text
            alerts.append(alert_text)
    return alerts


In [3]:
app = Flask(__name__)

@app.route('/get-alerts', methods=['GET'])
def get_alerts():
    line = request.args.get('line', '').upper()
    station = request.args.get('station', '').title()
    
    alerts = fetch_mta_alerts()
    if "error" in alerts:
        return jsonify(alerts), 500
    
    # Filter alerts by line and station if specified
    relevant_alerts = [alert for alert in alerts if line in alert and station in alert]
    
    return jsonify({"alerts": relevant_alerts})


In [4]:
app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Nov/2024 14:14:37] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [02/Nov/2024 14:14:37] "GET /favicon.ico HTTP/1.1" 404 -
